In [ ]:
from tiny_reader import TinyTrainReader
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization, ZeroPadding2D
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.utils import np_utils


In [ ]:
tiny_image_net_200_dir = '/src/dataset/imagenet/tiny-imagenet-200'
train_dir = os.path.join(tiny_image_net_200_dir, 'train')

reader = TinyTrainReader(train_dir)

In [ ]:
# show data
for idx, data in enumerate(reader.generate_data(1)):
    img = cv2.cvtColor((data[0][0] * 255).astype('uint8'), cv2.COLOR_RGB2BGR)
    plt.imshow(img)
    plt.show()
    break


In [ ]:
# Create model
model = Sequential()

#conv-spatial batch norm - relu #1 
model.add(ZeroPadding2D(padding = (2, 2), input_shape=(64, 64, 3)))
model.add(Conv2D(64, (5, 5)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #2
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #3
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #4
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #5
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #6
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #7
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#conv-spatial batch norm - relu #8
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(512, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#conv-spatial batch norm - relu #9
model.add(ZeroPadding2D(padding = (1,1)))
model.add(Conv2D(1024, (3, 3)))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))
model.add(MaxPooling2D())

#Affine-spatial batch norm - relu #10
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
model.add(Activation('relu'))

#Affine-spatial batch norm - softmax #11
model.add(Dense(reader.num_label))
model.add(Activation('softmax'))

# We add metrics to get more results you want to see
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

plot_model(model)

In [ ]:
'''data_generator = reader.generate_data(250)

model_check_point_callback = ModelCheckpoint("tiny_weights.{epoch:02d}-{acc:.2f}-{loss:.4f}.hdf5",
                                 monitor="loss",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="auto")

print('Training ------------')
model.fit_generator(data_generator, steps_per_epoch = 400, epochs=3, callbacks = [model_check_point_callback])'
'''

In [ ]:
x_train, y_train = reader.load_all_img()

In [ ]:
x_train = x_train / 255
y_train = np_utils.to_categorical(y_train, num_classes=reader.num_label)

In [ ]:
print('Training ------------')
model_check_point_callback = ModelCheckpoint("weights.{epoch:02d}-{acc:.2f}-{loss:.4f}.hdf5",
                                 monitor="loss",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="auto")
model.fit(x_train, y_train, epochs=100, batch_size=256, callbacks = [model_check_point_callback])